In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import pickle

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext autoreload
%autoreload 2
import opticalmaterials

# Glass catalog

In [ ]:
%%time
catalog = opticalmaterials.parse_rii_catalog(
    "compoundprisms/rii-database-2021-07-18.zip"
)
with open("compoundprisms/rii.pickle", "wb") as f:
    pickle.dump(catalog, f)

In [ ]:
%%time
with open("compoundprisms/rii.pickle", "rb") as f:
    catalog = pickle.load(f)

In [ ]:
catalog[("SCHOTT-LaK", "N-LAK12")]["DATA"]["formula 2"]

In [ ]:
n = opticalmaterials.get_n(catalog[("SCHOTT-LaK", "N-LAK12")])

In [ ]:
n(0.3)

In [ ]:
n(np.array([0.3, 1.2]))

In [ ]:
l = np.linspace(0.3, 1, 100)
plt.plot(l, n(l))

# Transmission

In [ ]:
df = catalog["N-SF11"]["DATA"]["tabulated k"]["data"]

In [ ]:
transmittance(df.index, df["kappa"], 10e3)

In [ ]:
df

In [ ]:
catalog["N-SF11"]["DATA"]["tabulated k"]["data"].hvplot()

In [ ]:
from bokeh.models import HoverTool

curves = []

thickness = 2e3  # in µm

for glass_name, glass in catalog.items():
    if not glass_name[1].endswith("HT"):
        continue
    if "tabulated k" not in glass["DATA"]:
        continue
    df = glass["DATA"]["tabulated k"]["data"]
    #     if df.index[0] > .28 or df.iloc[0,0] > 5e-6:
    #         continue
    T = opticalmaterials.transmittance(df.index.values, df["k"].values, thickness)
    tooltips = [("Name", "/".join(glass_name)), ("x", "@x"), ("y", "@y")]
    hover = HoverTool(tooltips=tooltips)
    curves.append(hv.Curve((df.index, T), label=glass_name).opts(tools=[hover]))

hv.Overlay(curves).redim.range(x=(0.24, 0.4)).opts(
    width=900, height=700, legend_position="bottom_right"
)

In [ ]:
z = glasses["schott_N-BK7"]["DATA"]["tabulated k"]["data"]

In [ ]:
z

In [ ]:
(z < 1e-7).idxmax()

In [ ]:
for glass in glasses.values():
    if "tabulated k" not in glass["DATA"]:
        continue
    df = glass["DATA"]["tabulated k"]["data"]
    T = transmittance(df.index, df["kappa"], thickness)

In [ ]:
T.index

In [ ]:
T.iloc[T.index.get_loc(1.2, method="nearest")]

In [ ]:
def _uv_transmittance(glass):
    thickness = 5e3  # µm
    lmbda = 0.3  # µm
    k = glass["DATA"]["tabulated k"]["data"]
    if k.index[0] > lmbda or not len(k):
        return 0
    T = pd.Series(opticalmaterials.transmittance(k.index.values, k["k"].values, thickness), index=k.index)
    idx = k.index.get_indexer([lmbda], method="nearest", tolerance=0.1)[0]
    return T.iloc[idx]


abbe_data = pd.DataFrame(
    list(
        [
            (
                "/".join(name),
                g["SPECS"]["Vd"],
                g["SPECS"]["nd"],
                # g["DATA"]["tabulated k"]["data"].index[0],
                (g["DATA"]["tabulated k"]["data"] < 5e-7).idxmax()[0],
                _uv_transmittance(g),
            )
            for name, g in catalog.items()
            if "SPECS" in g and "Vd" in g["SPECS"] and "tabulated k" in g["DATA"]
        ]
    ),
    columns=["name", "Vd", "nd", "UVcutoff", "T300_5mm"],
).set_index("name")

In [ ]:
abbe_data

In [ ]:
abbe_data[(abbe_data["T300_5mm"] > 0.7) & (abbe_data["T300_5mm"] != 1)].hvplot.scatter(
    "Vd",
    "nd",
    # color="UVcutoff",
    color="T300_5mm",
    cmap="spectral",
    use_index=True,
    hover_cols="all",
    flip_xaxis=True,
    width=600,
    height=500,
)

In [ ]:
abbe_data.hvplot.scatter(
    "Vd",
    "nd",
    # color="UVcutoff",
    color="T300_5mm",
    cmap="spectral",
    use_index=True,
    hover_cols="all",
    flip_xaxis=True,
    width=600,
    height=500,
)

In [ ]:
abbe_data[abbe_data.index.str.contains("ZF5")]

In [ ]:
uv_glasses = [
    "hoya_MC-TAF31-15",
    "hikari_J-LASF09A",
    "sumita_K-GIR140",
    "ohara_L-LAH87",
    "ohara_L-LAH91",
    "hikari_Q-LASFH11S",
    "cdgm_H-LAK61",
    "ohara_S-LAH97",
    "ohara_S-LAL19",
    "ohara_S-LAL18",
    "hoya_TAC8",
    "ohara_L-LAL15",
    "hikari_7054",
    "hikari_Q-LAK13S",
]
uv_glasses_maybe = ["cdgm_H-ZLAF69", "ohara_S-LAH59"]

In [ ]:
hv.Scatter()